In [13]:
import os
from pathlib import Path
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

dir_path = Path().parent.absolute()

# 데이터 경로를 정의합니다.
input_base = os.path.join(dir_path, "..", "data", "taxi")
# 파이프라인 컴포넌트를 인스턴스화합니다.
example_gen = CsvExampleGen(input_base=input_base)
# 대화식으로 컴포넌트를 실행합니다.
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/var/folders/8x/m76cd7zx24x2ph3x8k2b2r280000gn/T/tfx-interactive-2021-11-19T16_55_16.875377-6nhfphhr/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1922812,xor_checksum:1637308455,sum_checksum:1637308455"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [20]:
import os
from pathlib import Path
from tfx.components import ImportExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

dir_path = Path().parent.absolute()
tfrecord_dir = os.path.join(dir_path, "..", "..", "data", "tfrecord_data")

example_gen = ImportExampleGen(input_base= tfrecord_dir)

context.run(example_gen)

OSError: No files found based on the file pattern /Users/johanjun/Desktop/MLOps_study/building-machine-learning-pipelines/practices/../../data/tfrecord_data/*

In [21]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
from pathlib import Path

# 웹 상에서 원시 데이터를 다운 받습니다.
filepath = tf.keras.utils.get_file(
    "complaints.csv.zip",
    "http://files.consumerfinance.gov/ccdb/complaints.csv.zip")

dir_path = Path().parent.absolute()
data_dir = os.path.join(dir_path, "..", "data")
processed_dir = os.path.join(dir_path, "..", "data", "processed")
Path(processed_dir).mkdir(parents=True, exist_ok=True)

# 압축을 해제합니다.
shutil.unpack_archive(filepath, data_dir)
# pandas로 csv 파일을 읽어 들입니다.
df = pd.read_csv(os.path.join(data_dir, "complaints.csv"))

df.columns = [
    "date_received", "product", "sub_product", "issue", "sub_issue",
    "consumer_complaint_narrative", "company_public_response",
    "company", "state", "zip_code", "tags",
    "consumer_consent_provided", "submitted_via",
    "date_sent_to_company", "company_response",
    "timely_response", "consumer_disputed", "complaint_id"]

df.loc[df["consumer_disputed"] == "", "consumer_disputed"] = np.nan

# 주요한 필드가 비어있는 경우 레코드를 제외합니다.
df = df.dropna(subset=["consumer_complaint_narrative", "consumer_disputed"])

# Label 필드인 consumer_disputed를 Yes, No에서 1, 0 으로 변경합니다.
df.loc[df["consumer_disputed"] == "Yes", "consumer_disputed"] = 1
df.loc[df["consumer_disputed"] == "No", "consumer_disputed"] = 0

df.loc[df["zip_code"] == "", "zip_code"] = "000000"
df.loc[pd.isna(df["zip_code"]), "zip_code"] = "000000"

df = df[df['zip_code'].str.len() == 5]
df["zip_code"] = df['zip_code'].str.replace('XX', '00')
df = df.reset_index(drop=True)
df["zip_code"] = pd.to_numeric(df["zip_code"], errors='coerce')

# 판다스 DataFrame을 csv 파일로 다시 저장합니다.
df.to_csv(os.path.join(processed_dir, "processed-complaints.csv"), index=False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [23]:
import os
import re

import tensorflow as tf
import pandas as pd
from pathlib import Path


def _bytes_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode()])
    )


def _float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value])
    )


def _int64_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value])
    )


def clean_rows(row):
    if pd.isna(row["zip_code"]):
        row["zip_code"] = "99999"
    return row


def convert_zipcode_to_int(zipcode):
    nums = re.findall(r'\d+', zipcode)
    if len(nums) > 0:
        int_zipcode = int(nums[0])
    else:
        int_zipcode = 99999
    return int_zipcode

dir_path = Path().parent.absolute()
data_dir = os.path.join(dir_path, "..", "data")
tfrecord_dir = os.path.join(dir_path, "..", "data", "tfrecord")
df = pd.read_csv(os.path.join(data_dir, "processed/processed-complaints.csv"))

tfrecord_filename = "consumer-complaints.tfrecord"
tfrecord_filepath = os.path.join(tfrecord_dir, tfrecord_filename)
# tfrecord_filename에 지정된 경로에 저장하는 TFRecordWriter 객체를 만듭니다.
tf_record_writer = tf.io.TFRecordWriter(tfrecord_filepath)

for index, row in df.iterrows():
#     print(index)
    row = clean_rows(row)
    # 모든 데이터 레코드를 tf.train.Example로 변환
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "product": _bytes_feature(str(row["product"])),
                "sub_product": _bytes_feature(str(row["sub_product"])),
                "issue": _bytes_feature(str(row["issue"])),
                "sub_issue": _bytes_feature(str(row["sub_issue"])),
                "state": _bytes_feature(str(row["state"])),
                "zip_code": _int64_feature(convert_zipcode_to_int(str
                                                                  
                                                                  (row["zip_code"]))),
                "company": _bytes_feature(str(row["company"])),
                "company_response": _bytes_feature(str(row["company_response"])),
                "timely_response": _bytes_feature(str(row["timely_response"])),
                "consumer_disputed": _float_feature(
                    row["consumer_disputed"]
                ),
            }
        )
    )
    # 데이터 구조를 직렬화
    tf_record_writer.write(example.SerializeToString())
tf_record_writer.close()

In [123]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/johanjun/🔐/famous-analyzer-330314-cc4a230b0449.json"

In [124]:
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen

query = """
    SELECT * FROM `<bigquery_public_data>.<covid19_open_data_eu>.<covid19_open_data_eu>`
"""
example_gen = BigQueryExampleGen(query=query)

In [125]:
example_gen

BigQueryExampleGen(spec: <tfx.types.standard_component_specs.QueryBasedExampleGenSpec object at 0x7fbc9f460850>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7fbb0eb85880>, driver_class: <class 'tfx.components.example_gen.driver.QueryBasedDriver'>, component_id: BigQueryExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [130]:
os.path.join(os.getcwd(), "..")

'/Users/johanjun/Desktop/MLOps_study/practices/..'

In [25]:
import os
from pathlib import Path

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

context = InteractiveContext()

dir_path = Path().parent.absolute()
data_dir = os.path.join(dir_path, "..", "data", "processed")
output = example_gen_pb2.Output(
    # 선호하는 분할을 정의합니다.
    split_config=example_gen_pb2.SplitConfig(splits=[
        # 비율을 지정합니다.
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
        example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
    ]))

# output_config 인수를 추가합니다.
example_gen = CsvExampleGen(input_base=data_dir, output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/var/folders/8x/m76cd7zx24x2ph3x8k2b2r280000gn/T/tfx-interactive-2021-11-19T17_04_48.197905-ivlloynf/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:159960537,xor_checksum:1637308646,sum_checksum:1637308646"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [26]:
for artifact in example_gen.outputs['examples'].get():
    print(artifact)

Artifact(artifact: id: 1
type_id: 14
uri: "/var/folders/8x/m76cd7zx24x2ph3x8k2b2r280000gn/T/tfx-interactive-2021-11-19T17_04_48.197905-ivlloynf/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\", \"test\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:159960537,xor_checksum:1637308646,sum_checksum:1637308646"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.2.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_n